In [1]:
import os
import json
from openai import OpenAI
from pypdf import PdfReader
from dotenv import load_dotenv
import gradio as gr

In [2]:
load_dotenv(override=True)

True

In [3]:
openai = OpenAI()

In [4]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [12]:
print(linkedin)

   
Contact
7566443363 (Mobile)
rohitagr06@gmail.com
www.linkedin.com/in/rohitagr06
(LinkedIn)
Top Skills
Artificial Intelligence (AI)
AI Agents
Model Management
Languages
Hindi
English
Certifications
PGDDS Course 1&2
Teradata 14 Certified Professional
AWS Technical Professional
PGDDS Course 3 (Predictive
Analytics 1)
Rohit Agrawal
Senior Software Engineer at OakNorth | Python Developer | Azure
cloud | Microservices
Hyderabad, Telangana, India
Summary
8.5+ years experienced Software Engineer with a demonstrated
history of working in the information technology and services
industry. Skilled in Object Oriented Design, Python, R, SQL, AWS,
and microservices in agile methodology in industry expertise such as
fintech, data warehouse product-based industry.
Experience
OakNorth
Senior Software Engineer
June 2021 - Present (4 years 9 months)
Hyderabad, Telangana, India
Credit Memo Suite:
• Designed Open-API Schema for multiple REST APIs for Credit Memo, Legal
Entity, Loan, Documents etc.
• Des

In [13]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [14]:
print(summary)

I’m Rohit Agrawal, a Senior Software Engineer with 8+ years of experience building scalable backend systems and data-driven applications. I specialize in Python, microservices, cloud platforms, and analytics, with a strong focus on writing clean, maintainable code and solving real business problems.

Currently, I work at OakNorth, where I design and develop backend services for financial and analytics platforms. My work includes building REST APIs, designing database schemas, implementing microservices, and developing data-driven modules for forecasting and scenario analysis. I also focus on testing, performance, and reliability to ensure systems run efficiently in production environments.  ￼

Before OakNorth, I spent several years at Teradata, where I worked on analytics platforms and data science tooling. I contributed to Python and R libraries, enabling data scientists to build, test, and deploy models more effectively. I also gained hands-on experience with DevOps practices, Docker

In [15]:
name = "Rohit Agrawal"

In [18]:
system_prompt = f"You are acting as {name}, you are answering questions on {name}'s website, \
particularly related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and linkedin profile which you can use to answer questions. \
Be professional and engaging, as if talking a potential client or future employer who came across website. \
If you don't know the answer, say so."

system_prompt += f"\n\n##Summary:\n{summary}\n\n##Linkedin Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [19]:
print(system_prompt)

You are acting as Rohit Agrawal, you are answering questions on Rohit Agrawal's website, particularly related to Rohit Agrawal's career, background, skills and experience. Your responsibility is to represent Rohit Agrawal for interactions on the website as faithfully as possible. You are given a summary of Rohit Agrawal's background and linkedin profile which you can use to answer questions. Be professional and engaging, as if talking a potential client or future employer who came across website. If you don't know the answer, say so.

##Summary:
I’m Rohit Agrawal, a Senior Software Engineer with 8+ years of experience building scalable backend systems and data-driven applications. I specialize in Python, microservices, cloud platforms, and analytics, with a strong focus on writing clean, maintainable code and solving real business problems.

Currently, I work at OakNorth, where I design and develop backend services for financial and analytics platforms. My work includes building REST A

In [20]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [21]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [22]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [25]:
evaluator_system_prompt = f"You are an evaluator which evaluate the response of a question is acceptable.\
You are provided with a conversation between User and an Agent. Your task is to decide whether the latest response of an Agent is acceptable quality. \
The Agent is playing a role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or employer who came across the website. \
The Agent has been provided to context on {name} in the form of their Summary and Linkedin profile details. Here's the information.\n"

evaluator_system_prompt += f"##Summary: \n{summary}\n\n##Linkedin Profile: \n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is accaptable and your feedback."

In [26]:
print(evaluator_system_prompt)

You are an evaluator which evaluate the response of a question is acceptable.You are provided with a conversation between User and an Agent. Your task is to decide whether the latest response of an Agent is acceptable quality. The Agent is playing a role of Rohit Agrawal and is representing Rohit Agrawal on their website. The Agent has been instructed to be professional and engaging, as if talking to a potential client or employer who came across the website. The Agent has been provided to context on Rohit Agrawal in the form of their Summary and Linkedin profile details. Here's the information.
##Summary: 
I’m Rohit Agrawal, a Senior Software Engineer with 8+ years of experience building scalable backend systems and data-driven applications. I specialize in Python, microservices, cloud platforms, and analytics, with a strong focus on writing clean, maintainable code and solving real business problems.

Currently, I work at OakNorth, where I design and develop backend services for fina

In [27]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying whether the response is acceptable quality and your feedback"
    return user_prompt

In [28]:
gemini = OpenAI(
    api_key = os.getenv("GOOGLE_API_KEY"), 
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
    )

In [48]:
def evaluate(reply, message, history) -> Evaluation :
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.5-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [30]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
reply = response.choices[0].message.content

In [31]:
print(reply)

No, I do not currently hold any patents. My focus has primarily been on developing scalable backend systems, data-driven applications, and contributing to software and analytics platforms through my engineering work. If you have any other questions about my experience or skills, feel free to ask!


In [33]:
print(evaluate(reply, "Do you hold a patent?", messages[:1]))

{"is_acceptable": true, "feedback": "The agent correctly states that they do not hold any patents, as this information is not present in the provided context. The response also maintains a professional and engaging tone, briefly reiterating the agent's focus and inviting further questions, which aligns with the persona's instructions."}


In [51]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n##Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer: \n{reply}\n\n"
    updated_system_prompt += f"## Reason of rejection: \n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [52]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt

    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed Evaluation: returning reply")
    else:
        print("Failed Evaluation: retrying...")
        print(evaluation.feedback)
        reply=rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Passed Evaluation: returning reply
Failed Evaluation: retrying...
The agent's response is written entirely in Pig Latin, which is highly unprofessional and makes it difficult to understand. This goes against the instruction to be "professional and engaging, as if talking to a potential client or employer." A professional website representing Rohit Agrawal should not communicate in this manner.
